In [1]:
import pandas as pd
import openai
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GPT_API")
openai.api_key = api_key


def askChatGPT(messages):
    MODEL = "gpt-4"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=1)
    return response['choices'][0]['message']['content'] # type: ignore

In [11]:
msg_train_data = pd.read_csv("../data/CPED/train_split.csv")
msg_train_data = msg_train_data.iloc[0:200]
msg_train_data.head(2)

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,1,01_000,01_000_000,童文洁,female,middle-aged,high,high,low,low,high,other-venue,108_136,156_202,neutral,neutral,greeting,真巧
1,1,01_000,01_000_001,童文洁,female,middle-aged,high,high,low,low,high,other-venue,193_144,253_197,neutral,neutral,greeting,车没事了


# 标注内容

In [12]:

res = [{
    "role": "system",
    "content": "接下来，我将提供给你一段对话，对话的人数不确定。不过格式如下。'\n童文洁: 真巧\n童文洁: 车没事了\n刘静: 是你呀 没事了没事 谢谢你\n'。然后你需要提供对话人物的大五人格的维度属于高还是低。比如你需要回复，'童文洁: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 low\n刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'"
}] # 结果保存
for i in np.unique(msg_train_data["TV_ID"]):
    tmp_a = msg_train_data[msg_train_data["TV_ID"] == i] # 获取当前的对话ID所对应样本
    
    comm = ""
    gpt = ""
    for j, row in tmp_a.iterrows():
        comm += "{}: {}\n".format(row["Speaker"], row["Utterance"])
    res.append({
        "role": "user",
        "content": comm
    })
    for v in np.unique(tmp_a["Speaker"]):
        gpt += "{}: {}\n".format(v, "神经质 {}, 外倾性 {}, 开放性 {}, 亲和性 {}, 尽责性 {}".format(
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Neuroticism"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Extraversion"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Openness"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Agreeableness"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Conscientiousness"])[0]
        ))
    res.append({
        "role": "assistant",
        "content": gpt
    })
res

[{'role': 'system',
  'content': "接下来，我将提供给你一段对话，对话的人数不确定。不过格式如下。'\n童文洁: 真巧\n童文洁: 车没事了\n刘静: 是你呀 没事了没事 谢谢你\n'。然后你需要提供对话人物的大五人格的维度属于高还是低。比如你需要回复，'童文洁: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 low\n刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'"},
 {'role': 'user',
  'content': '童文洁: 真巧\n童文洁: 车没事了\n刘静: 是你呀 没事了没事 谢谢你\n童文洁: 没事没事 你也去春风\n刘静: 对\n童文洁: 你孩子也在这上学\n刘静: 对呀\n童文洁: 真巧 我儿子也在这\n童文洁: 你孩子几年级\n刘静: 高三了\n童文洁: 哪个班\n刘静: 三班啊\n童文洁: 我儿子也三班\n童文洁: 我儿子叫方一凡\n童文洁: 你孩子叫什么\n刘静: 我儿子叫季杨杨\n童文洁: 季杨杨，好像我听凡凡说过\n童文洁: 对\n童文洁: 是不是每次开家长会来的都是他爸爸\n刘静: 那是他舅舅\n童文洁: 我说呢这么年轻\n方一凡: 妈\n童文洁: 你不要叫我妈\n童文洁: 我不是你妈\n方一凡: 妈，我错了\n童文洁: 你没错\n童文洁: 我错了\n童文洁: 我为什么要生你\n童文洁: 我吃饱了撑的我就不该生你\n童文洁: 方一凡我真的不明白你脑子里面天天在想什么\n童文洁: 你是不是真的不知道现在高几了\n童文洁: 你不知道我告诉你\n童文洁: 你现在高三了\n童文洁: 不再是高一高二了\n童文洁: 你说\n童文洁: 你说你 天天天天\n童文洁: 学习不灵，打架门清，一个暑假我给你报个补习班\n童文洁: 原指望你好好学习能上去\n童文洁: 结果断崖式的下滑\n童文洁: 你对得起我吗\n童文洁: 妈妈高三那年妈妈怎么跟你讲的吗\n方一凡: 数学92\n方一凡: 化学91\n方一凡: 除了英语一般般\n方一凡: 其他都特别棒\n童文洁: 你为什么一点不随妈妈\n童文洁: 凡凡\n方一凡: 我随我爸爸\n童文洁: 你也没随你爸\n童文洁: 你爸也是学霸\n童文洁: 当年成绩也很好\n童文洁: 

# Test

In [10]:
msg_test_data = pd.read_csv("../data/CPED/test_split.csv")
msg_test_data = msg_test_data.iloc[0:200]
msg_test_data.head(2)

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,27,27_000,27_000_000,林宛瑜,female,young,low,high,high,high,high,car,488_14,654_204,neutral,neutral,statement-non-opinion,好了没事了
1,27,27_000,27_000_001,林宛瑜,female,young,low,high,high,high,high,car,481_47,613_244,neutral,neutral,thanking,谢谢你哦


In [13]:
for i in np.unique(msg_test_data["TV_ID"]):
    tmp_a = msg_test_data[msg_test_data["TV_ID"] == i] # 获取当前的对话ID所对应样本
    
    comm = ""
    gpt = ""
    for j, row in tmp_a.iterrows():
        comm += "{}: {}\n".format(row["Speaker"], row["Utterance"])
    res.append({
        "role": "user",
        "content": comm
    })
    for v in np.unique(tmp_a["Speaker"]):
        gpt += "{}: {}\n".format(v, "神经质 {}, 外倾性 {}, 开放性 {}, 亲和性 {}, 尽责性 {}".format(
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Neuroticism"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Extraversion"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Openness"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Agreeableness"])[0],
            np.unique(tmp_a[tmp_a["Speaker"] == v]["Conscientiousness"])[0]
        ))
    print({
        "role": "assistant",
        "content": gpt
    })
res

{'role': 'assistant', 'content': '其他: 神经质 unknown, 外倾性 unknown, 开放性 unknown, 亲和性 unknown, 尽责性 unknown\n吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low\n曾小贤: 神经质 high, 外倾性 high, 开放性 low, 亲和性 high, 尽责性 low\n林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n胡一菲: 神经质 low, 外倾性 high, 开放性 low, 亲和性 low, 尽责性 high\n陆展博: 神经质 high, 外倾性 low, 开放性 high, 亲和性 high, 尽责性 high\n'}


[{'role': 'system',
  'content': "接下来，我将提供给你一段对话，对话的人数不确定。不过格式如下。'\n童文洁: 真巧\n童文洁: 车没事了\n刘静: 是你呀 没事了没事 谢谢你\n'。然后你需要提供对话人物的大五人格的维度属于高还是低。比如你需要回复，'童文洁: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 low\n刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'"},
 {'role': 'user',
  'content': '童文洁: 真巧\n童文洁: 车没事了\n刘静: 是你呀 没事了没事 谢谢你\n童文洁: 没事没事 你也去春风\n刘静: 对\n童文洁: 你孩子也在这上学\n刘静: 对呀\n童文洁: 真巧 我儿子也在这\n童文洁: 你孩子几年级\n刘静: 高三了\n童文洁: 哪个班\n刘静: 三班啊\n童文洁: 我儿子也三班\n童文洁: 我儿子叫方一凡\n童文洁: 你孩子叫什么\n刘静: 我儿子叫季杨杨\n童文洁: 季杨杨，好像我听凡凡说过\n童文洁: 对\n童文洁: 是不是每次开家长会来的都是他爸爸\n刘静: 那是他舅舅\n童文洁: 我说呢这么年轻\n方一凡: 妈\n童文洁: 你不要叫我妈\n童文洁: 我不是你妈\n方一凡: 妈，我错了\n童文洁: 你没错\n童文洁: 我错了\n童文洁: 我为什么要生你\n童文洁: 我吃饱了撑的我就不该生你\n童文洁: 方一凡我真的不明白你脑子里面天天在想什么\n童文洁: 你是不是真的不知道现在高几了\n童文洁: 你不知道我告诉你\n童文洁: 你现在高三了\n童文洁: 不再是高一高二了\n童文洁: 你说\n童文洁: 你说你 天天天天\n童文洁: 学习不灵，打架门清，一个暑假我给你报个补习班\n童文洁: 原指望你好好学习能上去\n童文洁: 结果断崖式的下滑\n童文洁: 你对得起我吗\n童文洁: 妈妈高三那年妈妈怎么跟你讲的吗\n方一凡: 数学92\n方一凡: 化学91\n方一凡: 除了英语一般般\n方一凡: 其他都特别棒\n童文洁: 你为什么一点不随妈妈\n童文洁: 凡凡\n方一凡: 我随我爸爸\n童文洁: 你也没随你爸\n童文洁: 你爸也是学霸\n童文洁: 当年成绩也很好\n童文洁: 

In [14]:
askChatGPT(res)

'林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n陆展博: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n曾小贤: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n胡一菲: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 high\n吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low\n其他: 神经质 low, 外倾性 low, 开放性 low, 亲和性 low, 尽责性 low'

### 真实
吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low
曾小贤: 神经质 high, 外倾性 high, 开放性 low, 亲和性 high, 尽责性 low
林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
胡一菲: 神经质 low, 外倾性 high, 开放性 low, 亲和性 low, 尽责性 high
陆展博: 神经质 high, 外倾性 low, 开放性 high, 亲和性 high, 尽责性 high
其他: 神经质 unknown, 外倾性 unknown, 开放性 unknown, 亲和性 unknown, 尽责性 unknown

完了，我开始相信AI的标注了。因为爱情公寓第一季里面 曾小贤的扮演者 还没火，整体的感觉应该是 并不是那种 喜剧风格的神经质。

### GPT
吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low
曾小贤: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
胡一菲: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 high
陆展博: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
其他: 神经质 low, 外倾性 low, 开放性 low, 亲和性 low, 尽责性 low

In [18]:
tmp_a.groupby(["Speaker"])["Utterance"].count()

Speaker
其他     30
吕子乔    25
曾小贤    50
林宛瑜    25
胡一菲    37
陆展博    33
Name: Utterance, dtype: int64